In [ ]:
#kaart maken van de shit
#overzicht van alles, simpele metrics.

In [81]:
import pandas as pd

aed_locations = pd.read_csv('aed_locations_coordinates_processed.csv')
card_arrest = pd.read_csv('card_arrest_processed.csv')
all_hospital_locations = pd.read_csv('hospital_coordinates_processed.csv')
removed_aed = pd.read_csv('removed_AED.csv')

In [82]:
#during the data cleaning itself, 3576 AED were removed from the original 15227, equal to 23.48% of the original total. 2825 (18.55% of the total)
#because the coordinates could not be retrieved via Nomatim, for example because the provided address had abbreviations or misspellings.
#749 (4.91%) because they were considered to be unavailable to the public, and thus not useable.
#2 were removed simply because the address was missing.
removed_aed['reason'].value_counts()

reason
missing_coordinates    2825
not_available           749
address_missing           2
Name: count, dtype: int64

In [83]:
aed_locations.head()
#card_arrest.head()
#all_hospital_locations.head()
#missing_values_count_lat = all_hospital_locations['latitude'].isnull().sum()
#missing_values_count_long = all_hospital_locations['longitude'].isnull().sum()
#missing_values_count_lat
#missing_values_count_long

,address,number,municipality,latitude,longitude,id,postal_code,province
0,Place Saint - Lambert,0,Liège,50.750721,5.933967,71.0,4020.0,Liège
1,Rue du Doyard,0,Lierneux,50.285043,5.785950,72.0,4990.0,Liège
2,Fond Saint Servais,0,Liège,50.646765,5.570987,73.0,4000.0,Liège
3,Rue des Prémontrès,12,Liège,50.638047,5.574046,74.0,4020.0,Liège
4,Rue du Parc,1,Jehay,50.577233,5.323038,76.0,4540.0,Liège


In [84]:
#it could provide valuable insight to also plot the removed AED's. However, the vast majority of the AED's were removed because
#it was impossible to get coordinates in the first place. Only a small minority, for which it is clear what the reason is (they are not available)
#can be plotted, but since the reason is clear it is not really useful to plot them.
import folium

# approximately the center of Belgium coordinates
belgium_center_lat = 50.8503
belgium_center_lon = 4.3517

m = folium.Map(location=[belgium_center_lat, belgium_center_lon], zoom_start=8)

# add cardiac arrests to the map
for _, row in card_arrest.iterrows():
    folium.CircleMarker(
        location=[row['latitude_intervention'], row['longitude_intervention']],
        radius=1,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        tooltip='Cardiac Arrest'
    ).add_to(m)

# add AED locations to the map
for _, row in aed_locations.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.2,
        tooltip='AED'
    ).add_to(m)

# add hospital locations to the map
for _, row in all_hospital_locations.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2,
        tooltip='Hospital'
    ).add_to(m)

m.save('exploratory_map.html')

In [85]:
aed_locations

,address,number,municipality,latitude,longitude,id,postal_code,province
0,Place Saint - Lambert,0,Liège,50.750721,5.933967,71.0,4020.0,Liège
1,Rue du Doyard,0,Lierneux,50.285043,5.785950,72.0,4990.0,Liège
2,Fond Saint Servais,0,Liège,50.646765,5.570987,73.0,4000.0,Liège
3,Rue des Prémontrès,12,Liège,50.638047,5.574046,74.0,4020.0,Liège
4,Rue du Parc,1,Jehay,50.577233,5.323038,76.0,4540.0,Liège
...,...,...,...,...,...,...,...,...
11646,Farnières,4,Grand Halleux,50.310813,5.880418,16154.0,6698.0,Liège
11647,Chaussée de Marche,799,Wierde,50.426430,4.929522,16157.0,5100.0,Namur
11648,Nekkerspoel-Borcht,19,Mechelen,51.025217,4.504148,16158.0,2800.0,Antwerpen
11649,Nieuwe Dreef,17,Lokeren,51.097802,3.911744,16159.0,9160.0,Oost-Vlaanderen


In [86]:
#there are also some datapoints outside of belgium, especially for the AED's, AED's that got miscoded during the AED's location
#encoding using nomatim, likely because the address also existed in another country.
#we remove these instances by roughly taking the coordinates of the outermost edges of Belgium. Not all AED's that were miscoded (i.e. being outside
#of Belgium) get removed this way. 32 AED's get remove this way.

#add to the removed_aed data frame
incorrect_aed = aed_locations[
    (aed_locations['longitude'] <= 2.569241) | 
    (aed_locations['longitude'] >= 6.392983) |
    (aed_locations['latitude'] <= 49.479388) | 
    (aed_locations['latitude'] >= 51.556040)
]

incorrect_aed['reason'] = "incorrect_coordinates"

removed_aed = pd.concat([removed_aed, incorrect_aed])

aed_locations = aed_locations[aed_locations['longitude'] > 2.569241] #left
aed_locations = aed_locations[aed_locations['longitude'] < 6.392983] #right
aed_locations = aed_locations[aed_locations['latitude'] > 49.479388] #down
aed_locations = aed_locations[aed_locations['latitude'] < 51.556040] #up
aed_locations

C:\Users\eloua\AppData\Local\Temp\ipykernel_14948\1419426758.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_aed['reason'] = "incorrect_coordinates"


,address,number,municipality,latitude,longitude,id,postal_code,province
0,Place Saint - Lambert,0,Liège,50.750721,5.933967,71.0,4020.0,Liège
1,Rue du Doyard,0,Lierneux,50.285043,5.785950,72.0,4990.0,Liège
2,Fond Saint Servais,0,Liège,50.646765,5.570987,73.0,4000.0,Liège
3,Rue des Prémontrès,12,Liège,50.638047,5.574046,74.0,4020.0,Liège
4,Rue du Parc,1,Jehay,50.577233,5.323038,76.0,4540.0,Liège
...,...,...,...,...,...,...,...,...
11646,Farnières,4,Grand Halleux,50.310813,5.880418,16154.0,6698.0,Liège
11647,Chaussée de Marche,799,Wierde,50.426430,4.929522,16157.0,5100.0,Namur
11648,Nekkerspoel-Borcht,19,Mechelen,51.025217,4.504148,16158.0,2800.0,Antwerpen
11649,Nieuwe Dreef,17,Lokeren,51.097802,3.911744,16159.0,9160.0,Oost-Vlaanderen


In [ ]:
#een exactere manier om de AED's die buiten België liggen te verwijderen. als echt te veel tijd kan ik dit gebruiken.
"""
import geopandas as gpd
from shapely.geometry import Point

# Load Belgium's borders as a GeoDataFrame
belgium = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
belgium = belgium[belgium.name == 'Belgium']

# Assume df is your DataFrame with 'latitude' and 'longitude' columns.

# Convert the DataFrame's coordinates to a GeoSeries of Points
points = gpd.GeoSeries([Point(xy) for xy in zip(df['longitude'], df['latitude'])])

# Create a GeoDataFrame from your DataFrame, specifying the coordinate system
df_gdf = gpd.GeoDataFrame(df, geometry=points, crs="EPSG:4326")

# Perform a spatial join to filter points within Belgium
df_belgium = df_gdf[df_gdf.geometry.within(belgium.unary_union)]

# Drop the 'geometry' column if you don't need it anymore
df_belgium = df_belgium.drop(columns='geometry')

# df_belgium will now contain only the rows with coordinates inside Belgium.
"""

In [ ]:
# it is apparent that within cities the density of cases is highest, regardless of where
#the city is located. This makes sense since cities are simply more densely populated. The more Southern part of Wallonia has less densely 
#located cardiac arrests compared to the rest of the map, which also makes sense since it is less densely populated. In general it seems to 
#match the population counts of areas.
#Flanders is more densely populated, and the cardiac cases are more densely populated there.
#One last thing to notice is that there seems to be a belt of more densely spread cases which follows the
#Meuse river.

In [ ]:
#cardiac arrests per provincie
#aed's per provincie
#ziekenhuizen per provincie

#voor de cardiac arrests heb ik de province variabele verwijderd. dus dan zou ik terug moeten gaan in de data cleaning notebook en die kolom niet
#droppen (en voor de bxl card arrests "brussel" als value moeten geven ofzo).